### Canned Estimators How To

In this notebook we'll demonstrate how to use two Canned Estimators (these encapsulate the lower-level TensorFlow code we've seen so far, and use an API loosely inspired by [scikit-learn](scikit-learn.org). There are several advantages to Canned Estimators.

* If you're using Estimators, you won't have to manage Sessions, or write your own logic for TensorBoard, or for saving and loading checkpoints.

* You'll get out-of-the-box distributed training (of course, you will have to take care to read your data efficiently, and set up a cluster).

Here, we'll read data using [input functions](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn), which are appropriate for in-memory data. 

* These provide batching and other features for you, so you don't have to write that code yourself.
* In the structured data notebook, we'll use the [Dataset API](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/docs_src/programmers_guide/datasets.md) - which is a popular abstraction, and a great way to efficiently read and pre-process large datasets efficiently.

Although the Estimators we'll use here are relative simple (a LinearClassifier, and a Fully Connected Deep Neural Network), we also provide more interesting ones (including for [TensorFlow Wide and Deep](https://www.tensorflow.org/tutorials/wide_and_deep). I'm also excited that additional Estimators are on their way - stay tuned in the upcoming months.

Also note that Estimators log quite a lot of output

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

/Users/JosephNelson/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# We'll use Keras (included with TensorFlow) to import the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

# Normalize the color values to 0-1
# (as imported, they're 0-255)
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


Here's our input function. 

* By setting ```num_epochs``` to ```None```, we'll loop over the data indefinitely so we can train for as long as we like.
* The default ```batch_size``` is ```128```, but you can provide a different parameter if you like.

You can read more about the numpy input function [here](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/numpy_input_fn). One is on the way for [Pandas](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/pandas_input_fn).

In [3]:
train_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_train},
    y_train, 
    num_epochs=None, # repeat forever
    shuffle=True # 
)

In [4]:
test_input = tf.estimator.inputs.numpy_input_fn(
    {'x': x_test},
    y_test,
    num_epochs=1, # loop through the dataset once
    shuffle=False # don't shuffle the test data
)

In [5]:
# define the features for our model
# the names must match the input function
feature_spec = [tf.feature_column.numeric_column('x', shape=784)]

Here, we'll create a ```LinearClassifier``` - this is identical to Softmax (aka, multiclass logistic regression model).

In [6]:
estimator = tf.estimator.LinearClassifier(feature_spec, 
                                          n_classes=10,
                                          model_dir="./graphs/canned/linear")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './graphs/canned/linear', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1821887390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
# I've arbitrarily decided to train for 1000 steps
estimator.train(train_input, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/linear/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into ./graphs/canned/linear/model.ckpt.
INFO:tensorflow:loss = 40.047546, step = 1001
INFO:tensorflow:global_step/sec: 505.469
INFO:tensorflow:loss = 32.086037, step = 1101 (0.201 sec)
INFO:tensorflow:global_step/sec: 465.805
INFO:tensorflow:loss = 45.649136, step = 1201 (0.213 sec)
INFO:tensorflow:global_step/sec: 600.42
INFO:tensorflow:loss = 19.505938, step = 1301 (0.166 sec)
INFO:tensorflow:global_step/sec: 519.46
INFO:tensorflow:loss = 32.036507, step = 1401 (0.192 sec)
INFO:tensorflow:global_step/sec: 521.943
INFO:tensorflow:loss = 26.31308, step = 1501 (0.192 sec)
INFO:tensorflow:global_step/sec: 486.66
INFO:tensorflow:loss

In [8]:
# We should see about 90% accuracy here.
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-11-09:14:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/linear/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-11-09:14:08
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9239, average_loss = 0.27364606, global_step = 2000, loss = 34.638744
{'accuracy': 0.9239, 'average_loss': 0.27364606, 'loss': 34.638744, 'global_step': 2000}


Here's how you would print individual predictions.

In [9]:
MAX_TO_PRINT = 5

# This returns a generator object
predictions = estimator.predict(input_fn=test_input)
i = 0
for p in predictions:
    true_label = y_test[i]
    predicted_label = p['class_ids'][0]
    print("Example %d. True: %d, Predicted: %d" % (i, true_label, predicted_label))
    i += 1
    if i == MAX_TO_PRINT: break

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/linear/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Example 0. True: 7, Predicted: 7
Example 1. True: 2, Predicted: 2
Example 2. True: 1, Predicted: 1
Example 3. True: 0, Predicted: 0
Example 4. True: 4, Predicted: 4


Here's how easy it is to switch the model to a fully connected DNN.

In [10]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[256], # we will arbitrarily use two layers
    feature_columns=feature_spec,
    n_classes=10,
    model_dir="./graphs/canned/deep")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './graphs/canned/deep', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1831196ac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
# I've arbitrarily decided to train for 2000 steps
estimator.train(train_input, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/deep/model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4001 into ./graphs/canned/deep/model.ckpt.
INFO:tensorflow:loss = 3.905944, step = 4001
INFO:tensorflow:global_step/sec: 242.926
INFO:tensorflow:loss = 4.452634, step = 4101 (0.413 sec)
INFO:tensorflow:global_step/sec: 194.155
INFO:tensorflow:loss = 2.6407862, step = 4201 (0.515 sec)
INFO:tensorflow:global_step/sec: 252.06
INFO:tensorflow:loss = 3.3890014, step = 4301 (0.397 sec)
INFO:tensorflow:global_step/sec: 198.867
INFO:tensorflow:loss = 5.1384764, step = 4401 (0.504 sec)
INFO:tensorflow:global_step/sec: 264.406
INFO:tensorflow:loss = 1.609117, step = 4501 (0.377 sec)
INFO:tensorflow:global_step/sec: 234.037
INFO:tensorflow:loss = 2

In [12]:
# Expect accuracy around 97%
evaluation = estimator.evaluate(input_fn=test_input)
print(evaluation)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-11-09:14:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./graphs/canned/deep/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-11-09:14:18
INFO:tensorflow:Saving dict for global step 6000: accuracy = 0.9788, average_loss = 0.06775848, global_step = 6000, loss = 8.577023
{'accuracy': 0.9788, 'average_loss': 0.06775848, 'loss': 8.577023, 'global_step': 6000}


If you like, you can compare these runs with TensorBoard.

``` $ tensorboard --logdir=graphs/canned/ ```